In [14]:
import torch
import pandas as pd
import numpy as np

In [15]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [16]:
# read data here
# directory = "/home/tgbergendahl/research/NLP/data/archive/"
directory = "/Users/tgbergendahl/Research/data/archive/"
filename = "airbnb-listings.csv"
full_path = directory + filename

df = pd.read_csv(full_path, sep=';', nrows=10000)
x = len(df)
print(f"Read in {x} values")

Read in 10000 values


/var/folders/fm/tg2h9kb51zg6vym1k_4n9f_r0000gn/T/ipykernel_91893/1298179534.py:7: DtypeWarning: Columns (82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_path, sep=';', nrows=10000)


In [17]:
df = df[['ID', 'Summary', 'Price']]
df = df[df['Price']>0]
x = len(df)
print(f"Filtering out price=0 results in {x} values")
df = df[pd.notna(df['Summary'])]
x = len(df)
print(f"Filtering out N/A Summaries results in {x} values")

Filtering out price=0 results in 9793 values
Filtering out N/A Summaries results in 9496 values


In [18]:
df.head()

,ID,Summary,Price
0,4008728,‪This luxurious apartment is situated in the c...,600.0
1,7778612,"Beautiful, large (105m2, 2 floors) and quiet a...",175.0
2,8264596,This central located apartment will make you f...,125.0
3,2180729,Living like an Amsterdam resident in this ligh...,130.0
4,14463171,Studio Apartment Centre Amsterdam Perfect stud...,80.0


In [19]:
mean_len = df['Summary'].apply(len).mean()

mean_len

281.8249789385004

In [20]:
def get_mse(x, y):
    return (x-y)**2

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

In [22]:
from sklearn.model_selection import train_test_split

x = df['Summary']
y = df['Price']

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

x_train_vectorized = vectorizer.fit_transform(x_train)
x_test_vectorized = vectorizer.transform(x_test)

In [23]:
from sklearn.neural_network import MLPRegressor

regr = MLPRegressor(random_state=42, max_iter=2500).fit(x_train_vectorized, y_train)

In [24]:
regr.score(x_test_vectorized, y_test)

-0.35076203560105945

In [25]:
test_pred = regr.predict(x_test_vectorized)

res_df = pd.DataFrame({
    'Predictions': test_pred,
    'Ground Truth': y_test
})

res_df.head(20)

,Predictions,Ground Truth
7759,141.157403,64.0
2310,27.167511,38.0
3985,197.175415,120.0
3152,148.773440,40.0
999,128.002097,159.0
9277,129.101254,69.0
9561,917.654228,240.0
7364,47.496299,25.0
4732,602.611271,450.0
9228,65.404557,70.0


In [26]:
res_df['mse'] = res_df.apply(lambda row: get_mse(row['Predictions'], row['Ground Truth']), axis=1)

np.mean(res_df['mse'])

34398.66797019605